In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
import pickle

In [3]:
from fuzzysearch import find_near_matches

In [4]:
df = pd.read_csv("/home/jujun/fraudprediction_10k/df_10k_20200202.csv")

In [5]:
df.shape

(67993, 37)

In [6]:
df.head()

,Unnamed: 0,Unnamed: 0.1,file,CIK,type,yeara,fraud_match,SIC_fl_rp,save_path_7,num_para,...,ib,ivao,dlc,sstk,dltis,pstk,lt,ivst,cogs,dp
0,3,3,edgar/data/798757/0001035704-99-000038.txt,798757,10-k,1998,0,7372.0,/research/jujun/saveitem_1128/item7/798757-000...,70,...,74.468,322.527,0.0,47.824,0.0,0.000,366.477,28.667,400.552,22.701
1,4,4,edgar/data/798757/0000950134-00-000532.txt,798757,10-k,1999,0,7372.0,/research/jujun/saveitem_1128/item7/798757-000...,100,...,-39.224,246.564,0.0,31.568,0.0,0.000,347.808,62.546,453.871,25.421
2,5,5,edgar/data/798757/0000950109-02-006127.txt,798757,10-k,2002,0,7372.0,/research/jujun/saveitem_1128/item7/798757-000...,124,...,46.193,17.203,0.0,16.705,0.0,0.000,367.768,0.301,350.605,31.264
3,6,6,edgar/data/798757/0001035704-01-000019.txt,798757,10-k,2000,0,7372.0,/research/jujun/saveitem_1128/item7/798757-000...,108,...,-15.422,107.458,0.0,41.829,0.0,89.113,390.930,49.434,441.998,31.039
4,7,7,edgar/data/798757/0001035704-02-000070.txt,798757,10-k,2001,0,7372.0,/research/jujun/saveitem_1128/item7/798757-000...,110,...,-179.753,0.702,0.0,25.962,0.0,0.000,362.073,1.515,412.687,31.910


In [7]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'file', 'CIK', 'type', 'yeara',
       'fraud_match', 'SIC_fl_rp', 'save_path_7', 'num_para', 'num_words',
       'gvkey_x', 'map_key', 'gvkey_y', 'SIC', 'datadate', 'fyear', 'conm',
       'che', 'rect', 'invt', 'act', 'lct', 'at', 'ppent', 'dltt', 'sale',
       'ib', 'ivao', 'dlc', 'sstk', 'dltis', 'pstk', 'lt', 'ivst', 'cogs',
       'dp'],
      dtype='object')

In [8]:
df_1 = df.loc[:,['file','save_path_7','gvkey_x','gvkey_y','fyear','conm']]

In [9]:
df_1

,file,save_path_7,gvkey_x,gvkey_y,fyear,conm
0,edgar/data/798757/0001035704-99-000038.txt,/research/jujun/saveitem_1128/item7/798757-000...,65466.0,65466.0,1998.0,EDWARDS J D & CO
1,edgar/data/798757/0000950134-00-000532.txt,/research/jujun/saveitem_1128/item7/798757-000...,65466.0,65466.0,1999.0,EDWARDS J D & CO
2,edgar/data/798757/0000950109-02-006127.txt,/research/jujun/saveitem_1128/item7/798757-000...,65466.0,65466.0,2002.0,EDWARDS J D & CO
3,edgar/data/798757/0001035704-01-000019.txt,/research/jujun/saveitem_1128/item7/798757-000...,65466.0,65466.0,2000.0,EDWARDS J D & CO
4,edgar/data/798757/0001035704-02-000070.txt,/research/jujun/saveitem_1128/item7/798757-000...,65466.0,65466.0,2001.0,EDWARDS J D & CO
5,edgar/data/798757/0001035704-98-000029.txt,/research/jujun/saveitem_1128/item7/798757-000...,65466.0,65466.0,1997.0,EDWARDS J D & CO
6,edgar/data/914174/0000016918-00-000009.txt,/research/jujun/saveitem_1128/item7/914174-000...,2710.0,2710.0,1999.0,CONSTELLATION BRANDS
7,edgar/data/914174/0000016918-99-000008.txt,/research/jujun/saveitem_1128/item7/914174-000...,2710.0,2710.0,1998.0,CONSTELLATION BRANDS
8,edgar/data/914174/0000016918-98-000001.txt,/research/jujun/saveitem_1128/item7/914174-000...,2710.0,2710.0,1997.0,CONSTELLATION BRANDS
9,edgar/data/914174/0000016918-97-000006.txt,/research/jujun/saveitem_1128/item7/914174-000...,2710.0,2710.0,1996.0,CONSTELLATION BRANDS


In [10]:
name_list = []
for n in df_1.conm:
    n_temp = n.split(' ')
    name_list += n_temp

In [11]:
import collections

In [12]:
name_counter=collections.Counter(name_list)

In [13]:
type(name_counter)

collections.Counter

In [14]:
name_counter = dict(name_counter)

In [15]:
name_tb = pd.DataFrame.from_dict(name_counter, orient='index',columns = ['counts'] )

In [16]:
name_tb = name_tb.reset_index().rename(columns = {'index':'words'})

In [17]:
name_tb = name_tb.sort_values(by=['counts'],ascending = False)

In [18]:
name_tb.head(15)

,words,counts
5729,INC,40746
5719,CORP,14869
8149,CO,3095
573,GROUP,2801
0,,2169
6813,TECHNOLOGIES,2123
5842,HOLDINGS,1944
4715,A,1769
4703,-CL,1721
5420,&,1679


In [19]:
df_1[df_1.conm.str.contains('-CL')]

,file,save_path_7,gvkey_x,gvkey_y,fyear,conm
297,edgar/data/1124610/0001193125-08-043667.txt,/research/jujun/saveitem_1128/item7/1124610-00...,178083.0,178083.0,2007.0,VMWARE INC -CL A
298,edgar/data/1124610/0001124610-15-000005.txt,/research/jujun/saveitem_1128/item7/1124610-00...,178083.0,178083.0,2014.0,VMWARE INC -CL A
299,edgar/data/1124610/0001445305-12-000443.txt,/research/jujun/saveitem_1128/item7/1124610-00...,178083.0,178083.0,2011.0,VMWARE INC -CL A
300,edgar/data/1124610/0001124610-16-000040.txt,/research/jujun/saveitem_1128/item7/1124610-00...,178083.0,178083.0,2015.0,VMWARE INC -CL A
301,edgar/data/1124610/0001124610-14-000008.txt,/research/jujun/saveitem_1128/item7/1124610-00...,178083.0,178083.0,2013.0,VMWARE INC -CL A
302,edgar/data/1124610/0001124610-17-000009.txt,/research/jujun/saveitem_1128/item7/1124610-00...,178083.0,178083.0,2016.0,VMWARE INC -CL A
303,edgar/data/1124610/0001124610-13-000009.txt,/research/jujun/saveitem_1128/item7/1124610-00...,178083.0,178083.0,2012.0,VMWARE INC -CL A
304,edgar/data/1024047/0001012870-03-001176.txt,/research/jujun/saveitem_1128/item7/1024047-00...,64186.0,64186.0,2002.0,LARSCOM INC -CL A
305,edgar/data/1024047/0000891618-04-000818.txt,/research/jujun/saveitem_1128/item7/1024047-00...,64186.0,64186.0,2003.0,LARSCOM INC -CL A
306,edgar/data/1024047/0001047469-98-012294.txt,/research/jujun/saveitem_1128/item7/1024047-00...,64186.0,64186.0,1997.0,LARSCOM INC -CL A


In [20]:
rm_list = ['inc','corp','co','group','holdings','-cl','a','b']

In [21]:
def match_name(x, rm_list):
    x = x.lower()
    wrds = x.split(' ')
    m_list = []
    for w in wrds:
        if w not in rm_list:
            m_list.append(w)
    return ' '.join(m_list)
    

In [22]:
df_1['name_1'] = df_1.conm.apply(lambda x: match_name(x, rm_list))

In [23]:
def clean_name(x):
    t = x.lower().split()
    result1 = t[0]
    result2 = None
    result3 = None
    if len(t)>1:
        result2=' '.join(t[0:2])
        if len(t)>2:
            result3=' '.join(t[0:3])
    return result1, result2, result3

In [24]:
df_1['name_2'] = df_1.name_1.apply(lambda x: clean_name(x))

In [25]:
df_1.head()

,file,save_path_7,gvkey_x,gvkey_y,fyear,conm,name_1,name_2
0,edgar/data/798757/0001035704-99-000038.txt,/research/jujun/saveitem_1128/item7/798757-000...,65466.0,65466.0,1998.0,EDWARDS J D & CO,edwards j d &,"(edwards, edwards j, edwards j d)"
1,edgar/data/798757/0000950134-00-000532.txt,/research/jujun/saveitem_1128/item7/798757-000...,65466.0,65466.0,1999.0,EDWARDS J D & CO,edwards j d &,"(edwards, edwards j, edwards j d)"
2,edgar/data/798757/0000950109-02-006127.txt,/research/jujun/saveitem_1128/item7/798757-000...,65466.0,65466.0,2002.0,EDWARDS J D & CO,edwards j d &,"(edwards, edwards j, edwards j d)"
3,edgar/data/798757/0001035704-01-000019.txt,/research/jujun/saveitem_1128/item7/798757-000...,65466.0,65466.0,2000.0,EDWARDS J D & CO,edwards j d &,"(edwards, edwards j, edwards j d)"
4,edgar/data/798757/0001035704-02-000070.txt,/research/jujun/saveitem_1128/item7/798757-000...,65466.0,65466.0,2001.0,EDWARDS J D & CO,edwards j d &,"(edwards, edwards j, edwards j d)"


In [26]:
def load_data(data_name):
    f = open(data_name,'rb')
    data = pickle.load(f)
    return data

In [27]:
df_data = load_data('/home/jujun/fraudprediction_10k/data/data_20200212_2012')

In [44]:
def rm_name(d, names):
    
    d_1 = '/n/t/r '.join(d)
    for n in names[::-1]: # start with the first 3 words in the name
        #print('\n', n)
        if n!= None:  
            match = find_near_matches(n, d_1, max_l_dist= int(0.1*len(n)))
            #print(match)
            if len(match) > 0:
                new_d = ''
                start = 0
                for m in match: 
                    new_d = new_d + d_1[start:m.start] 
                    start = m.end
                new_d = new_d+ d_1[start:]
                d_fl = new_d.split('/n/t/r')
            else: d_fl = d
            
    #print('finish')            
    return d_fl

In [ ]:
c =0
data_rmn = []
for item in df_data:
    c += 1 
    if c % 10000 == 0:
        print(c)
    d = item[0]
    file = item[-1]
    label = item[1]
    cik = item[2]
    names_idx = df_1[df_1.file == file].index
    #print(type(int(names_idx[0])))
    names = df_1.loc[int(names_idx[0])]["name_2"]
    #print(names)
    d_fl = rm_name(d, names)
    new_item = (d_fl,label,cik,file)
    data_rmn.append(new_item)

In [ ]:
with open('data_20200309_2012_rmname', 'wb') as fp:
     pickle.dump(data_rmn, fp)
fp.close()